# Conway's Game of Life

Om de voorbeelden te kunnen zien moet je eerst klikken op 'Cell' ->  'Run All'

# John Conway

... was een Britse wiskundige. Hij bedacht 'Game of Life' in 1970.

Conway overleed op 11 april 2020 ten gevolge van COVID-19.

Zijn andere verdiensten zijn minder bekend, maar zeker de moeite om eens te bestuderen.
Een kleine selectie:

* [Surreële getallen](https://www.youtube.com/watch?v=mPn2AdMH7UQ)
* [De Monster groep](https://www.youtube.com/watch?v=jsSeoGpiWsw)
* [De Rij van Conway en Conway's constante](https://www.youtube.com/watch?v=ea7lJkEhytA)

# Game of Life

* Wordt gespeeld op een (oneindig) tweedimensionaal raster.
* Het is een spel voor 0 personen.
* Het is een voorbeeld van een 'cellulaire automaat'.
    - Elk vak in het raster wordt een cel genoemd.
    - Iedere cel heeft een toestand, bijvoorbeeld 'Aan' of 'Uit'.
    - De toestand van de cel evolueert op basis van de huidige toestand, en de toestand van zijn buren.

Het blokje code hieronder bevat een uitgewerkte versie van 'Game of Life'.

Je kan dit even overslagen, en doorbladeren naar de spel-regels.

In [1]:
import copy
import random
import time

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from IPython.display import display
import ipywidgets as widgets
import time

class GameOfLife:

    def __init__(self, grid=None):
        self.grid = grid

    @classmethod
    def random(cls, lines=1, rows=1, seed=1):
        random.seed(seed)
        grid = []
        grid = [None] * lines

        for y in range(0, lines):
            grid[y] = list(random.choice([True, False]) for i in range(rows))
            
        return cls(grid)
    
    @classmethod
    def fromImage(cls, file):
        img = mpimg.imread(file)
        grid = []
        grid = [None] * len(img)
        
        for y in range(0, len(img)):
            grid[y] = list(map(lambda pixel: True if pixel[0] == 0 else False, img[y]))
        
        return cls(grid)

    def number_of_neighbours(self, y, x):
        # <x>
        # xxx    ^
        # x x    y
        # xxx    v

        coordinates = [
          (y-1, x-1),
          (y-1, x),
          (y-1, x+1),

          (y, x-1),
          (y, x+1),

          (y+1, x-1),
          (y+1, x),
          (y+1, x+1)
        ]
        positive_coordinates = list(filter(lambda point: point[0] >= 0 and point[1] >= 0, coordinates))
        positive_coordinates = list(filter(lambda point: point[0] < len(self.grid) and point[1] < len(self.grid[0]), positive_coordinates))

        neighbours = list(map(lambda point: self.grid[point[0]][point[1]], positive_coordinates))
        return len(list(filter(lambda cell: cell is True, neighbours)))

    def tick(self):
        new_grid = copy.deepcopy(self.grid)
        for y in range(0, len(self.grid)):
            row = self.grid[y]
            for x in range(0, len(row)):
                cell = row[x]
                number_of_neighbours = self.number_of_neighbours(y, x)
                # 1. Any live cell with two or three neighbours survives
                if cell is True and (number_of_neighbours == 2 or number_of_neighbours == 3):
                    new_grid[y][x] = True
                # 2. Any dead cell with three live neighbours becomes a live cell:
                elif cell is False and (number_of_neighbours == 3):
                    new_grid[y][x] = True
                # 3. All other live cells die in the next generation.
                else:
                    new_grid[y][x] = False

        self.grid = new_grid
        
    def loop(self, iterations, output):
        output.clear_output()
        with output:
            out = widgets.HTML()
            display(out)

            for i in range(0, iterations):
                out.value = self.generateHtml()

                time.sleep(0.250)
                self.tick()
                
    def generateHtml(self):
        table = '<table class="game-of-life" style="border: none; padding: 0 0 0 0;">'

        aliveCell = '<td height="15px" width="15px" style="background:black;"></td>'
        deadCell  = '<td height="15px" width="15px" style="border:none;"></td>'
        for y in range(0, len(self.grid)):
            table += '<tr>'
            row = self.grid[y]
            row = list(map(lambda cell: aliveCell if cell is True else deadCell, row))
            table += ''.join(row)
            table += '</tr>'

        table += '</table>'
        return table
                
    def show(self, iterations=10):
        button = widgets.Button(
            description='Start',
            disabled=False,
            tooltip='Start',
            icon='play'
        )

        output = widgets.Output()
       
        def on_button_clicked(b):
            self.loop(iterations, output)

        display(button)
        display(output)
        
        with output:
            out = widgets.HTML()
            display(out)
            out.value = self.generateHtml()
        button.on_click(on_button_clicked)

In [2]:
game = GameOfLife.random(20, 20, 1)
game.show(30)

Button(description='Start', icon='play', style=ButtonStyle(), tooltip='Start')

Output()

## Regels

Een cel heeft twee toestanden: 'Dood' of 'Levend'. In onze voorstelling zijn dode cellen wit gekleurd (zelfde kleur als de achtergrond, dus onzichtbaar). Levende cellen zijn zwart gekleurd.

Om te weten wat de volgende toestand van een cel is, moeten we volgende regels volgen:

1. Een levende cell met 2 of 3 buren overleeft;
2. Een dode cell met 3 buren wordt levend;
3. Alle andere cellen gaan dood.

Een aantal patronen komen regelmatig terug. We tonen voor elke categorie een voorbeeldje.

Probeer bij elk voorbeeld de regels van het spel toe te passen. Dit lukt het best door de start figuur over te nemen op een blad papier. Teken een tweede - leeg - raster, en bepaal de toestand voor elke cel.

Meer patronen vind je op: [https://en.wikipedia.org/wiki/Conway's_Game_of_Life#Examples_of_patterns](https://en.wikipedia.org/wiki/Conway's_Game_of_Life#Examples_of_patterns)

## Oscillator

Na een aantal iteraties verschijnt opnieuw de start-figuur. In dit voorbeeld is de periode gelijk aan '1'.

In [3]:
game = GameOfLife([[False, False, False], [True, True, True], [False, False, False]])
game.show(10)

Button(description='Start', icon='play', style=ButtonStyle(), tooltip='Start')

Output()

## Stilleven

Bij een stilleven blijven de cellen in dezelfde toestand. (Dit is geen bug)

In [4]:
game = GameOfLife([[False, False, False, False], [False, True, True, False], [False, True, True, False], [False, False, False, False]])
game.show(10)

Button(description='Start', icon='play', style=ButtonStyle(), tooltip='Start')

Output()

## Ruimteschip

Een ruimteschip is een figuur die zich over het raster verplaatst.
Omdat de computer geen oneindig geheugen heeft, verdwijnt het ruimteschip wanneer het de rand bereikt.
Mocht je een oneindig raster hebben dan zou het in theorie ook oneindig lang door blijven bewegen.

In [5]:
game = GameOfLife.fromImage('spaceship.png')
game.show(100)

Button(description='Start', icon='play', style=ButtonStyle(), tooltip='Start')

Output()

# Een cool patroon om af te sluiten

Er bestaan verschillende interessante patronen. Het onderstaande is een klassieker.
Het genereert ruimteschepen!

In [6]:
game = GameOfLife.fromImage("goslin_glider.png")
game.show(100)

Button(description='Start', icon='play', style=ButtonStyle(), tooltip='Start')

Output()